In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import requests
import json
import time
import numpy as np
from yelpapi import YelpAPI
from pprint import pprint
from API import api_key

In [2]:
# creat the code to get the data from yelp 

url = 'https://api.yelp.com/v3/businesses/search'
API = api_key
headers = {'Authorization': 'bearer %s' % API}
offset = 0
business = []


# loop throught the URL to get 500 rows of data 

for i in range(0, 500, 50):
    params = {'location': 'Cleveland, Ohio' , 'offset': offset , 'limit': 50 }
    response = requests.get(url, params=params, headers=headers)
    business.append(response.json())
    offset += 50
    
    if offset % 50 == 0:
        time.sleep(5)

        


In [3]:
business[1]['businesses']

[{'id': 'cUwNW5NEua_yMcAx-X2sMg',
  'alias': 'grumpys-cafe-cleveland',
  'name': "Grumpy's Cafe",
  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/pxoH6ePSvagPvG9ckRv-Zg/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/grumpys-cafe-cleveland?adjust_creative=TshdU4XV3-ETjsHU3-ICtw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=TshdU4XV3-ETjsHU3-ICtw',
  'review_count': 328,
  'categories': [{'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'},
   {'alias': 'salad', 'title': 'Salad'}],
  'rating': 4.0,
  'coordinates': {'latitude': 41.476519, 'longitude': -81.690993},
  'transactions': ['delivery'],
  'price': '$$',
  'location': {'address1': '2621 W 14th St',
   'address2': '',
   'address3': '',
   'city': 'Cleveland',
   'zip_code': '44113',
   'country': 'US',
   'state': 'OH',
   'display_address': ['2621 W 14th St', 'Cleveland, OH 44113']},
  'phone': '+12162415025',
  'display_phone': '(216) 241-5025',
  'distance': 1589.909540

In [4]:
# creat a loop that will open all the 10 json files togeather

full_data = []
for data in business:
    full_data = full_data + data["businesses"]
    


len(full_data)

500

In [5]:
# convert 10 json files to a DF

df = pd.DataFrame.from_dict(full_data)
df

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,lola-cleveland,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...","{'latitude': 41.49883, 'longitude': -81.69035}",(216) 621-5652,3191.079270,J1RDyyPxhioqm8c_fi4P4Q,https://s3-media1.fl.yelpcdn.com/bphoto/XuGT-E...,False,"{'address1': '2058 E 4th St', 'address2': '', ...",Lola,+12166215652,$$$,4.0,980,[],https://www.yelp.com/biz/lola-cleveland?adjust...
1,townhall-cleveland-2,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 41.485492, 'longitude': -81.703914}",(216) 344-9400,2951.385340,LNsZJP6jZ11e0tDljOLPiQ,https://s3-media4.fl.yelpcdn.com/bphoto/ycqFgv...,False,"{'address1': '1909 W 25th St', 'address2': '',...",Townhall,+12163449400,$$,4.0,991,[],https://www.yelp.com/biz/townhall-cleveland-2?...
2,l-albatros-cleveland,"[{'alias': 'french', 'title': 'French'}]","{'latitude': 41.5117439550152, 'longitude': -8...",(216) 791-7880,6953.412053,SzcCJJmreMlWCsNjupdVAA,https://s3-media4.fl.yelpcdn.com/bphoto/QpVBtp...,False,"{'address1': '11401 Bellflower Rd', 'address2'...",L'Albatros,+12167917880,$$$,4.5,604,[],https://www.yelp.com/biz/l-albatros-cleveland?...
3,slymans-restaurant-cleveland,"[{'alias': 'tradamerican', 'title': 'American ...","{'latitude': 41.51286, 'longitude': -81.67125}",(216) 621-3760,4387.852814,1veVZUawy7IhIc5oDpRRQA,https://s3-media2.fl.yelpcdn.com/bphoto/rz2ST-...,False,"{'address1': '3106 Saint Clair Ave NE', 'addre...",Slyman's Restaurant,+12166213760,$$,4.5,417,[],https://www.yelp.com/biz/slymans-restaurant-cl...
4,sokolowskis-university-inn-cleveland,"[{'alias': 'polish', 'title': 'Polish'}, {'ali...","{'latitude': 41.48475, 'longitude': -81.69005}",(216) 771-9236,1942.420138,Xny0n0s98TpP82sQxfgIMQ,https://s3-media2.fl.yelpcdn.com/bphoto/F1jEYU...,False,"{'address1': '1201 University Rd', 'address2':...",Sokolowski's University Inn,+12167719236,$$,4.5,464,[],https://www.yelp.com/biz/sokolowskis-universit...
5,great-lakes-brewing-company-cleveland-4,"[{'alias': 'breweries', 'title': 'Breweries'},...","{'latitude': 41.4844102, 'longitude': -81.7043...",(216) 771-4404,2944.089481,meXjqyhTNLFmknY39y2sMg,https://s3-media1.fl.yelpcdn.com/bphoto/RHNLaz...,False,"{'address1': '2516 Market Ave', 'address2': ''...",Great Lakes Brewing Company,+12167714404,$$,4.0,850,[],https://www.yelp.com/biz/great-lakes-brewing-c...
6,momocho-cleveland-2,"[{'alias': 'mexican', 'title': 'Mexican'}]","{'latitude': 41.4840151, 'longitude': -81.7098...",(216) 694-2122,3334.932184,F2pfjAZ_3dMTGCKv6c5wOw,https://s3-media3.fl.yelpcdn.com/bphoto/D61sVw...,False,"{'address1': '1835 Fulton Rd', 'address2': Non...",Momocho,+12166942122,$$,4.0,694,[],https://www.yelp.com/biz/momocho-cleveland-2?a...
7,luckys-cafe-cleveland-3,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...","{'latitude': 41.47736, 'longitude': -81.68308}",(216) 622-7773,992.894340,_5PJ4GHIXNdUdXtohylKGQ,https://s3-media4.fl.yelpcdn.com/bphoto/-AYXJl...,False,"{'address1': '777 Starkweather Ave', 'address2...",Lucky's Cafe,+12166227773,$$,4.0,771,[],https://www.yelp.com/biz/luckys-cafe-cleveland...
8,happy-dog-cleveland,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...","{'latitude': 41.4848327636719, 'longitude': -8...",(216) 651-9474,4685.922534,anzCdD5eIEHwydTpXhLlUA,https://s3-media2.fl.yelpcdn.com/bphoto/2r3m49...,False,"{'address1': '5801 Detroit Ave', 'address2': '...",Happy Dog,+12166519474,$,4.0,605,[],https://www.yelp.com/biz/happy-dog-cleveland?a...
9,superior-pho-cleveland,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...","{'latitude': 41.51056, 'longitude': -81.6698}",(216) 781-7462,4127.849575,AbPQf-X7awuPFDULiJ43bg,https://s3-media2.fl.yelpcdn.com/bphoto/Vb88rq...,False,"{'address1': '3030 Superior Ave E', 'address2'...",Superior Pho,+12167817462,$,4.0,580,[pickup],https://www.yelp.com/biz/superior-pho-clevelan...


In [12]:
# clean any dic col. in this loop

idx = 0
lats = []
lngs = []
Bus_titles = []
ZIPS = []
Trans = []
for i, r in df.iterrows():
    lat = r["coordinates"]["latitude"]
    lng = r["coordinates"]["longitude"]
    titles = r["categories"][0]["title"]
    ZIP = r["location"]["zip_code"]
    if len(r['transactions']) == 0:
        Trans = "0"
    else:
        Trans = "1"
        
    #print(title)
    #print(lat,lng)
    lats.append(lat)
    lngs.append(lng)
    Bus_titles.append(titles)
    ZIPS.append(ZIP)
    #print(Trans)



    
#print(lats)
#print(lngs)
#DF = df.insert(loc=6, column='A', value=Bus_titles)
#df

In [16]:
# an DF that only have the cleaned Cols.

clean_df = pd.DataFrame(data={'Categories': Bus_titles, 
                              'latitude': lats,
                              'longitude': lngs,
                              'zip_code': ZIPS,
                              'Delivery/No_Delivery': Trans })
clean_df.head()

,Categories,latitude,longitude,zip_code,Delivery/No_Delivery
0,Wine Bars,41.498830,-81.690350,44115,0
1,American (New),41.485492,-81.703914,44113,0
2,French,41.511744,-81.606395,44106,0
3,American (Traditional),41.512860,-81.671250,44114,0
4,Polish,41.484750,-81.690050,44113,0


In [17]:
# adding the cleaned col to the original DF

conat_df = pd.concat([df,clean_df], axis=1 )


# Drop un needed Dic col that have been cleaned from the oringinal DF

drop_df=conat_df.drop(columns = ['coordinates', 'categories'])
drop_df

,alias,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,Categories,latitude,longitude,zip_code,Delivery/No_Delivery
0,lola-cleveland,(216) 621-5652,3191.079270,J1RDyyPxhioqm8c_fi4P4Q,https://s3-media1.fl.yelpcdn.com/bphoto/XuGT-E...,False,"{'address1': '2058 E 4th St', 'address2': '', ...",Lola,+12166215652,$$$,4.0,980,[],https://www.yelp.com/biz/lola-cleveland?adjust...,Wine Bars,41.498830,-81.690350,44115,0
1,townhall-cleveland-2,(216) 344-9400,2951.385340,LNsZJP6jZ11e0tDljOLPiQ,https://s3-media4.fl.yelpcdn.com/bphoto/ycqFgv...,False,"{'address1': '1909 W 25th St', 'address2': '',...",Townhall,+12163449400,$$,4.0,991,[],https://www.yelp.com/biz/townhall-cleveland-2?...,American (New),41.485492,-81.703914,44113,0
2,l-albatros-cleveland,(216) 791-7880,6953.412053,SzcCJJmreMlWCsNjupdVAA,https://s3-media4.fl.yelpcdn.com/bphoto/QpVBtp...,False,"{'address1': '11401 Bellflower Rd', 'address2'...",L'Albatros,+12167917880,$$$,4.5,604,[],https://www.yelp.com/biz/l-albatros-cleveland?...,French,41.511744,-81.606395,44106,0
3,slymans-restaurant-cleveland,(216) 621-3760,4387.852814,1veVZUawy7IhIc5oDpRRQA,https://s3-media2.fl.yelpcdn.com/bphoto/rz2ST-...,False,"{'address1': '3106 Saint Clair Ave NE', 'addre...",Slyman's Restaurant,+12166213760,$$,4.5,417,[],https://www.yelp.com/biz/slymans-restaurant-cl...,American (Traditional),41.512860,-81.671250,44114,0
4,sokolowskis-university-inn-cleveland,(216) 771-9236,1942.420138,Xny0n0s98TpP82sQxfgIMQ,https://s3-media2.fl.yelpcdn.com/bphoto/F1jEYU...,False,"{'address1': '1201 University Rd', 'address2':...",Sokolowski's University Inn,+12167719236,$$,4.5,464,[],https://www.yelp.com/biz/sokolowskis-universit...,Polish,41.484750,-81.690050,44113,0
5,great-lakes-brewing-company-cleveland-4,(216) 771-4404,2944.089481,meXjqyhTNLFmknY39y2sMg,https://s3-media1.fl.yelpcdn.com/bphoto/RHNLaz...,False,"{'address1': '2516 Market Ave', 'address2': ''...",Great Lakes Brewing Company,+12167714404,$$,4.0,850,[],https://www.yelp.com/biz/great-lakes-brewing-c...,Breweries,41.484410,-81.704395,44113,0
6,momocho-cleveland-2,(216) 694-2122,3334.932184,F2pfjAZ_3dMTGCKv6c5wOw,https://s3-media3.fl.yelpcdn.com/bphoto/D61sVw...,False,"{'address1': '1835 Fulton Rd', 'address2': Non...",Momocho,+12166942122,$$,4.0,694,[],https://www.yelp.com/biz/momocho-cleveland-2?a...,Mexican,41.484015,-81.709813,44113,0
7,luckys-cafe-cleveland-3,(216) 622-7773,992.894340,_5PJ4GHIXNdUdXtohylKGQ,https://s3-media4.fl.yelpcdn.com/bphoto/-AYXJl...,False,"{'address1': '777 Starkweather Ave', 'address2...",Lucky's Cafe,+12166227773,$$,4.0,771,[],https://www.yelp.com/biz/luckys-cafe-cleveland...,Coffee & Tea,41.477360,-81.683080,44113,0
8,happy-dog-cleveland,(216) 651-9474,4685.922534,anzCdD5eIEHwydTpXhLlUA,https://s3-media2.fl.yelpcdn.com/bphoto/2r3m49...,False,"{'address1': '5801 Detroit Ave', 'address2': '...",Happy Dog,+12166519474,$,4.0,605,[],https://www.yelp.com/biz/happy-dog-cleveland?a...,Bars,41.484833,-81.726547,44102,0
9,superior-pho-cleveland,(216) 781-7462,4127.849575,AbPQf-X7awuPFDULiJ43bg,https://s3-media2.fl.yelpcdn.com/bphoto/Vb88rq...,False,"{'address1': '3030 Superior Ave E', 'address2'...",Superior Pho,+12167817462,$,4.0,580,[pickup],https://www.yelp.com/biz/superior-pho-clevelan...,Vietnamese,41.510560,-81.669800,44114,0


In [19]:
# Display the new DF with a spacific col for better

display_df = drop_df[['name','Categories','zip_code','latitude','longitude','Delivery/No_Delivery','rating','review_count','price','location']]
display_df

,name,Categories,zip_code,latitude,longitude,Delivery/No_Delivery,rating,review_count,price,location
0,Lola,Wine Bars,44115,41.498830,-81.690350,0,4.0,980,$$$,"{'address1': '2058 E 4th St', 'address2': '', ..."
1,Townhall,American (New),44113,41.485492,-81.703914,0,4.0,991,$$,"{'address1': '1909 W 25th St', 'address2': '',..."
2,L'Albatros,French,44106,41.511744,-81.606395,0,4.5,604,$$$,"{'address1': '11401 Bellflower Rd', 'address2'..."
3,Slyman's Restaurant,American (Traditional),44114,41.512860,-81.671250,0,4.5,417,$$,"{'address1': '3106 Saint Clair Ave NE', 'addre..."
4,Sokolowski's University Inn,Polish,44113,41.484750,-81.690050,0,4.5,464,$$,"{'address1': '1201 University Rd', 'address2':..."
5,Great Lakes Brewing Company,Breweries,44113,41.484410,-81.704395,0,4.0,850,$$,"{'address1': '2516 Market Ave', 'address2': ''..."
6,Momocho,Mexican,44113,41.484015,-81.709813,0,4.0,694,$$,"{'address1': '1835 Fulton Rd', 'address2': Non..."
7,Lucky's Cafe,Coffee & Tea,44113,41.477360,-81.683080,0,4.0,771,$$,"{'address1': '777 Starkweather Ave', 'address2..."
8,Happy Dog,Bars,44102,41.484833,-81.726547,0,4.0,605,$,"{'address1': '5801 Detroit Ave', 'address2': '..."
9,Superior Pho,Vietnamese,44114,41.510560,-81.669800,0,4.0,580,$,"{'address1': '3030 Superior Ave E', 'address2'..."
